In [27]:
import requests
import plotly.graph_objects as go
import pandas as pd
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict, MessageToJson
from datetime import datetime


# ensure to have a key.txt file with your API key in it
with open('key.txt') as f:
    apiKey = f.read().strip()

# set the headers for the request
headers = {'api_key': apiKey}

In [28]:
# importing static data

# get the static data
staticData = requests.get('https://api.wmata.com/gtfs/rail-gtfs-static.zip', headers=headers)

with open('static.zip', 'wb') as f:
    f.write(staticData.content)


In [29]:

# recieving the response of the API vehicle position data
feed = gtfs_realtime_pb2.FeedMessage()

response  = requests.get('https://api.wmata.com/gtfs/rail-gtfsrt-vehiclepositions.pb', headers=headers)
feed.ParseFromString(response.content)

currentdata = MessageToDict(feed)

print(currentdata)


{'header': {'gtfsRealtimeVersion': '2.0', 'incrementality': 'FULL_DATASET', 'timestamp': '1739227999'}, 'entity': [{'id': '0', 'isDeleted': False, 'vehicle': {'trip': {'tripId': '6573199_20121', 'startTime': '17:56:00', 'startDate': '20250210', 'scheduleRelationship': 'SCHEDULED', 'routeId': 'RED', 'directionId': 0}, 'position': {'latitude': 39.119896, 'longitude': -77.164734, 'bearing': 148.0}, 'currentStopSequence': 1, 'currentStatus': 'STOPPED_AT', 'timestamp': '1739227999', 'stopId': 'PF_A15_C', 'vehicle': {'id': '120', 'label': '127', 'licensePlate': '6_3246-3247.3183-3182.3195-3194'}, 'occupancyStatus': 'EMPTY'}}, {'id': '1', 'isDeleted': False, 'vehicle': {'trip': {'tripId': '6573158_20121', 'startTime': '17:57:00', 'startDate': '20250210', 'scheduleRelationship': 'SCHEDULED', 'routeId': 'RED', 'directionId': 1}, 'position': {'latitude': 39.061802, 'longitude': -77.0535, 'bearing': 136.0}, 'currentStopSequence': 1, 'currentStatus': 'STOPPED_AT', 'timestamp': '1739227999', 'stopI

In [30]:
MessageToJson(feed)

'{\n  "header": {\n    "gtfsRealtimeVersion": "2.0",\n    "incrementality": "FULL_DATASET",\n    "timestamp": "1739227999"\n  },\n  "entity": [\n    {\n      "id": "0",\n      "isDeleted": false,\n      "vehicle": {\n        "trip": {\n          "tripId": "6573199_20121",\n          "startTime": "17:56:00",\n          "startDate": "20250210",\n          "scheduleRelationship": "SCHEDULED",\n          "routeId": "RED",\n          "directionId": 0\n        },\n        "position": {\n          "latitude": 39.119896,\n          "longitude": -77.164734,\n          "bearing": 148.0\n        },\n        "currentStopSequence": 1,\n        "currentStatus": "STOPPED_AT",\n        "timestamp": "1739227999",\n        "stopId": "PF_A15_C",\n        "vehicle": {\n          "id": "120",\n          "label": "127",\n          "licensePlate": "6_3246-3247.3183-3182.3195-3194"\n        },\n        "occupancyStatus": "EMPTY"\n      }\n    },\n    {\n      "id": "1",\n      "isDeleted": false,\n      "vehi

In [31]:
# Extracting the relevant data
entities = currentdata['entity']
parsed_data = []

for entity in entities:
    vehicle = entity['vehicle']
    trip = vehicle['trip']
    position = vehicle['position']
    vehicle_info = vehicle['vehicle']
    
    parsed_data.append({
        'id': entity['id'],
        'isDeleted': entity['isDeleted'],
        'tripId': trip['tripId'],
        'startTime': datetime.strptime(trip['startTime'], '%H:%M:%S').time(),
        'startDate': datetime.strptime(trip['startDate'], '%Y%m%d').date(),
        'scheduleRelationship': trip['scheduleRelationship'],
        'routeId': trip['routeId'],
        'directionId': trip['directionId'],
        'latitude': position['latitude'],
        'longitude': position['longitude'],
        'bearing': position['bearing'],
        'currentStopSequence': vehicle.get('currentStopSequence'),
        'currentStatus': vehicle.get('currentStatus'),
        'timestamp': datetime.fromtimestamp(int(vehicle['timestamp'])),
        'stopId': vehicle.get('stopId'),
        'vehicleId': vehicle_info['id'],
        'label': vehicle_info['label'],
        'licensePlate': vehicle_info['licensePlate'],
        'occupancyStatus': vehicle.get('occupancyStatus')
    })

# Creating DataFrame
GTFSVehiclePosition = pd.DataFrame(parsed_data)
GTFSVehiclePosition.head()


,id,isDeleted,tripId,startTime,startDate,scheduleRelationship,routeId,directionId,latitude,longitude,bearing,currentStopSequence,currentStatus,timestamp,stopId,vehicleId,label,licensePlate,occupancyStatus
0,0,False,6573199_20121,17:56:00,2025-02-10,SCHEDULED,RED,0,39.119896,-77.164734,148.0,1.0,STOPPED_AT,2025-02-10 22:53:19,PF_A15_C,120,127,6_3246-3247.3183-3182.3195-3194,EMPTY
1,1,False,6573158_20121,17:57:00,2025-02-10,SCHEDULED,RED,1,39.061802,-77.053500,136.0,1.0,STOPPED_AT,2025-02-10 22:53:19,PF_B11_C,469,102,8_7744-7745.7095-7094.7036-7037.7507-7506,EMPTY
2,2,False,6555688_20121,18:01:00,2025-02-10,SCHEDULED,ORANGE,0,38.877415,-77.273766,78.0,1.0,STOPPED_AT,2025-02-10 22:53:19,PF_K08_C,118,947,8_7696-7697.7003-7002.7714-7715.7473-7472,EMPTY
3,3,False,6555617_20121,17:59:00,2025-02-10,SCHEDULED,SILVER,0,39.007660,-77.493720,146.0,1.0,STOPPED_AT,2025-02-10 22:53:19,PF_N12_C,470,647,6_7306-7307.7332-7333.7595-7594,EMPTY
4,4,False,6555641_20121,17:54:00,2025-02-10,SCHEDULED,SILVER,1,38.900555,-76.844710,226.0,1.0,STOPPED_AT,2025-02-10 22:53:19,PF_G05_C,411,602,8_7148-7149.7153-7152.7277-7276.7301-7300,EMPTY


In [32]:
# recieving the response of the API trip update data
feed2 = gtfs_realtime_pb2.FeedMessage()
response2  = requests.get('https://api.wmata.com/gtfs/rail-gtfsrt-tripupdates.pb', headers=headers)
feed2.ParseFromString(response2.content)

print(MessageToJson(feed2))

{
  "header": {
    "gtfsRealtimeVersion": "2.0",
    "incrementality": "FULL_DATASET",
    "timestamp": "1739228000"
  },
  "entity": [
    {
      "id": "0",
      "tripUpdate": {
        "trip": {
          "tripId": "6573199_20121",
          "startTime": "17:56:00",
          "startDate": "20250210",
          "scheduleRelationship": "SCHEDULED",
          "routeId": "RED",
          "directionId": 0
        },
        "stopTimeUpdate": [
          {
            "stopSequence": 1,
            "departure": {
              "time": "1739228160",
              "uncertainty": 0
            },
            "stopId": "PF_A15_C",
            "scheduleRelationship": "SCHEDULED"
          },
          {
            "stopSequence": 2,
            "arrival": {
              "time": "1739228350",
              "uncertainty": 0
            },
            "stopId": "PF_A14_C",
            "scheduleRelationship": "SCHEDULED"
          },
          {
            "stopSequence": 3,
            "arri

In [33]:
routeShapes = pd.read_csv('static/shapes.csv')
routeShapes = routeShapes[routeShapes['shape_id'].isin(['RRED_16', 'RGRN_72', 'RYEL_96', 'RBLU_47', 'RSLV_192', 'RORG_134'])]
shape_id_mapping = {
    'RRED_16': 'RED',
    'RGRN_72': 'GREEN',
    'RYEL_96': 'YELLOW',
    'RBLU_47': 'BLUE',
    'RSLV_192': 'SILVER',
    'RORG_134': 'ORANGE'
}

routeShapes['shape_id'] = routeShapes['shape_id'].map(shape_id_mapping)
routeShapes


,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
5590,RED,39.061819,-77.053576,1,0.0000
5591,RED,39.061803,-77.053597,2,0.0015
5592,RED,39.060027,-77.051422,3,0.1716
5593,RED,39.059622,-77.050958,4,0.2091
5594,RED,39.059230,-77.050593,5,0.2426
...,...,...,...,...,...
74675,SILVER,38.896839,-76.849606,708,43.4055
74676,SILVER,38.897077,-76.849141,709,43.4352
74677,SILVER,38.897420,-76.848653,710,43.4703
74678,SILVER,38.898845,-76.846845,711,43.6078


In [34]:
stops = pd.read_csv('static/stops.csv')
stops

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,location_type,parent_station,wheelchair_boarding,level_id
0,STN_N06,WIEHLE-RESTON EAST METRORAIL STATION,NaN,38.947832,-77.340316,91,1,NaN,1,NaN
1,STN_N04,SPRING HILL METRORAIL STATION,NaN,38.929212,-77.241891,90,1,NaN,1,NaN
2,STN_N03,GREENSBORO METRORAIL STATION,NaN,38.921269,-77.234162,89,1,NaN,1,NaN
3,STN_N02,TYSONS METRORAIL STATION,NaN,38.920685,-77.221589,88,1,NaN,1,NaN
4,STN_N01,MCLEAN METRORAIL STATION,NaN,38.924339,-77.210530,87,1,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...
2170,NODE_C11_MZ_ESC2_TP,POTOMAC YARD-VT METRORAIL STATION,ESCALATOR/STAIR TO TRACK 2 PLATFORM,38.833957,-77.046516,98,3,STN_C11,0,C11_L2
2171,NODE_C11_MZ_ESC2_BT,POTOMAC YARD-VT METRORAIL STATION,TRACK 2 PLATFROM LEVEL ESCALATOR/STAIR TO MEZZ...,38.833748,-77.046520,98,3,STN_C11,0,C11_L1
2172,NODE_C11_MZ_ESC1_TP,POTOMAC YARD-VT METRORAIL STATION,ESCALATOR/STAIR TO TRACK 1 PLATFORM,38.833967,-77.046281,98,3,STN_C11,0,C11_L2
2173,NODE_C11_MZ_ESC1_BT,POTOMAC YARD-VT METRORAIL STATION,TRACK 1 PLATFROM LEVEL ESCALATOR/STAIR TO MEZZ...,38.833747,-77.046280,98,3,STN_C11,0,C11_L1


In [35]:
fig = go.Figure()

# Plot route shapes
for shape_id in routeShapes['shape_id'].unique():
    shape_data = routeShapes[routeShapes['shape_id'] == shape_id]
    fig.add_trace(go.Scattermap(lat=shape_data['shape_pt_lat'], lon=shape_data['shape_pt_lon'], mode='lines', name=shape_id))

# Plot vehicle positions
fig.add_trace(go.Scattermap(
    lat=GTFSVehiclePosition['latitude'],
    lon=GTFSVehiclePosition['longitude'],
    mode='markers',
    marker=dict(size=8, color='red'),
    name='Vehicle Positions'
))

fig.update_layout(
    map=dict(
        zoom=10,
        center=dict(lat=38.9, lon=-77.05),
        style='dark'
    )
)

fig.show()

In [36]:
# def getRealTime():
#     # !!! ADD STOP ARRIVAL INFORMATION (POPUPS AND DATA FROM TRIPS.JSON)
#     # !!! WILL NEED TO FIND STOP_ID IN TRIPS.JSON AND STOP_TIMES.TXT
#     # FIND MATCHING STOP_ID / TRIP_ID SEQUENCE ADD NEW ARRIVAL TIME FROM STOP_TIMES.TXT

#     def parseDict(pbu):
#         # TAKES THE DATA FROM U (THE PB URL) AND TURNS IT INTO A DICTIONARY
#         feed = gtfs_realtime_pb2.FeedMessage()
#         url = pbu
#         response = requests.get(url, headers=headers)
#         feed.ParseFromString(response.content)
#         feed2 = MessageToDict(feed)
#         return feed2

#     def getVehicles(pburl):
#         def addVehicleInfo(vehicles):
#             with open(r'leaflet\routes.json') as data:
#                 routesJson = json.loads(data.read())
#                 for vehicle in vehicles['features']:
#                     routei = 0

#                     # FINDING A ROUTE_ID MATCH BETWEEN VEHICLES AND ROUTESJSON. CYCLES THROUGH ROUTES TO FIND A MATCH
#                     # !!! ASSUMES THAT THERE WILL BE A MATCH. FACTOR IN A NO MATCH BY LOOKING FOR END OF ROUTES LIST.
#                     while vehicle['data']['routeId'] != routesJson[routei]['route_id']:
#                         routei += 1

#                     # IF ROUTE_ID MATCH FOUND
#                     if vehicle['data']['routeId'] != routesJson[routei]['route_id']:
#                         routei = 0

#                     # COPY OVER ROUTESJSON ROUTE_SHORT_NAME AND ROUTE_LONG_NAME TO VEHICLES
#                     vehicle['data']['route_short_name'] = routesJson[routei]['route_short_name']
#                     vehicle['data']['route_long_name'] = routesJson[routei]['route_long_name']
#                     vehicle['data']['route_full_name'] = routesJson[routei]['route_short_name'] + " " + routesJson[routei]['route_long_name']

#                 return vehicles

#         def addVehiclePopups(vehicles):
#             for vehicle in vehicles['features']:
#                 vehicle["properties"]["popupContent"] = f"Route: {vehicle['data']['route_short_name']} " \
#                                                         f"<br>Route Name: {vehicle['data']['route_long_name']} " \
#                                                         f"<br>TripID: {vehicle['data']['tripId']} " \
#                                                         f"<br>VehicleID: {vehicle['data']['vehicleId']}"
#             return vehicles

#         allVehicles = {}
#         allVehicles['type'] = {}
#         allVehicles['type'] = 'Feature Collection'
#         allVehicles['features'] = []

#         feed = parseDict(pburl)
#         id = 0
#         for value in feed['entity']:
#             obj = {}

#             # LIST OF SECTIONS
#             list = ["type", "properties", "geometry", "data"]
#             for i in list: # CREATE SECTIONS
#                 obj[i] = {}
#             obj["type"] = "Feature"

#             # START OF DATA SECTION
#             tripId = value["vehicle"]["trip"]["tripId"]
#             uni = obj["data"]
#             uni["vehicleId"] = value["vehicle"]["vehicle"]["id"]
#             uni["tripId"] = tripId
#             uni["routeId"] = value["vehicle"]["trip"]["routeId"]
#             uni["coordinates"] = [value["vehicle"]["position"]["longitude"], value["vehicle"]["position"]["latitude"]]

#             # START OF GEOMETRY SECTION
#             obj["geometry"]["type"] = "Point"
#             obj["geometry"]["coordinates"] = uni["coordinates"]


#             # START OF PROPERTIES SECTION
#             obj["properties"] = {}
#             obj["properties"]['id'] = id

#             # ADD INDIVIDUAL VEHICLES TO LIST
#             id += 1
#             allVehicles['features'].append(obj)
#         allVehicles = addVehicleInfo(allVehicles)
#         allVehicles = addVehiclePopups(allVehicles)
#         return allVehicles

#     def getTrips(pburl):
#         allTrips = {}
#         feed = parseDict(pburl)
#         for value in feed['entity']:
#             tripId = value['tripUpdate']['trip']['tripId']
#             allTrips[tripId] = {}
#             allTrips[tripId]['tripId'] = tripId
#             allTrips[tripId]['routeId'] = value['tripUpdate']['trip']['routeId']
#             if 'delay' in value['tripUpdate']['stopTimeUpdate'][0]['departure']:
#                 allTrips[tripId]['delay'] = value['tripUpdate']['stopTimeUpdate'][0]['departure']['delay']
#             if 'delay' in value['tripUpdate']['stopTimeUpdate'][0]['departure']:
#                 allTrips[tripId]['time'] = value['tripUpdate']['stopTimeUpdate'][0]['departure']['time']
#             if 'delay' in value['tripUpdate']['stopTimeUpdate'][0]['departure']:
#                 allTrips[tripId]['nextStopId'] = value['tripUpdate']['stopTimeUpdate'][0]['stopId']

#         return allTrips


#     realtime_list = [
#         'https://www.metrostlouis.org/RealTimeData/StlRealTimeVehicles.pb',
#         'https://www.metrostlouis.org/RealTimeData/StlRealTimeTrips.pb'
#     ]

#     for item in realtime_list:

#         # if looking at vehicles
#         if item == 'https://www.metrostlouis.org/RealTimeData/StlRealTimeVehicles.pb':
#             print('writing vehicles...')
#             vehicles = getVehicles(item)
#             saveTempData(vehicles, r'leaflet\vehicles.json')
#             pass
#         # if looking at the trips file
#         elif item == 'https://www.metrostlouis.org/RealTimeData/StlRealTimeTrips.pb':
#             print('writing trips...')
#             print(item)
#             trips = getTrips(item)
#             saveTempData(trips, r'leaflet\trips.json')
#             pass
#         else:
#             print(item)
#             print('error')
#             return